#Data Initialization

In [1]:
import random
import copy
random.seed(10)
Data=[]
for i in range(10000):
   Data.append(random.randint(0,7))

print(Data)
print(len(Data))

[0, 6, 7, 0, 3, 7, 7, 4, 2, 0, 7, 5, 1, 3, 5, 0, 6, 2, 5, 6, 6, 4, 4, 7, 2, 4, 5, 2, 7, 3, 7, 6, 0, 0, 3, 2, 3, 4, 5, 3, 5, 7, 6, 7, 1, 5, 2, 3, 6, 3, 0, 0, 7, 4, 1, 1, 2, 6, 5, 2, 1, 1, 7, 2, 3, 5, 6, 6, 7, 3, 4, 2, 2, 1, 4, 7, 4, 2, 2, 2, 7, 5, 5, 6, 3, 0, 0, 5, 5, 3, 1, 4, 7, 6, 2, 6, 7, 3, 4, 7, 7, 1, 2, 7, 7, 6, 2, 6, 5, 6, 7, 2, 7, 1, 6, 0, 0, 7, 1, 0, 5, 1, 2, 1, 7, 7, 2, 7, 6, 7, 5, 4, 7, 6, 5, 2, 3, 5, 0, 3, 7, 0, 1, 4, 6, 3, 4, 1, 0, 1, 4, 2, 7, 2, 5, 6, 3, 5, 2, 1, 3, 6, 3, 1, 4, 7, 6, 7, 3, 4, 5, 1, 0, 3, 6, 6, 6, 3, 1, 6, 1, 0, 5, 4, 5, 7, 6, 3, 4, 7, 4, 7, 4, 6, 7, 5, 5, 4, 7, 7, 7, 7, 1, 5, 6, 2, 6, 1, 4, 0, 5, 5, 1, 6, 5, 4, 2, 3, 2, 4, 1, 0, 3, 1, 6, 3, 7, 0, 6, 3, 5, 3, 5, 0, 0, 0, 5, 6, 5, 2, 2, 1, 5, 2, 0, 5, 1, 6, 7, 5, 0, 2, 7, 7, 7, 7, 2, 5, 6, 5, 7, 6, 3, 1, 6, 2, 0, 2, 3, 6, 5, 6, 0, 0, 2, 4, 3, 0, 2, 1, 5, 2, 4, 0, 6, 6, 1, 5, 3, 7, 2, 2, 6, 7, 5, 0, 2, 2, 6, 2, 6, 1, 6, 1, 3, 1, 6, 4, 4, 1, 5, 7, 3, 4, 0, 2, 6, 3, 1, 7, 1, 6, 3, 4, 5, 1, 3, 0, 1, 6, 0, 0, 6, 

In [2]:
def match_(arr1, arr2):
  length= min(len(arr1),len(arr2))
  for i in range(length):
    if arr1[i] != arr2[i]:
      return False
  return True

#New CRC

In [4]:
!pip install crc==4.1.0    # crc 4.1.0

In [5]:
from crc import Calculator, Configuration
import math

config = Configuration(
    width=12,
    polynomial=0xd31,
    init_value=0x000,
    final_xor_value=0xfff,
    reverse_input=False,
    reverse_output=False,
)


class DigitRecover:
    def __init__(self, data_blck_size, csum_blck_size, data_arr_size, csum_arr_size, top_k, Depth_Limit):
        #self.crc_calculator = CrcCalculator(Crc8.CCITT, True)
        self.calculator = Calculator(config)
        self.data_blck_size = data_blck_size
        self.csum_blck_size = csum_blck_size
        self.data_arr_size = data_arr_size
        self.csum_arr_size = csum_arr_size
        self.top_k = top_k




    def gen_idx(self, n_d, n_c):
        if n_d == 0:
            return [[]]
        ret = []
        for c in range(n_c):
            for arr in self.gen_idx(n_d - 1, n_c):
                ret.append([c] + arr)
        return ret

    def rank(self, digits, confs):
        batch_size = self.data_blck_size + self.csum_blck_size
        arr_idx = self.gen_idx(batch_size, self.top_k)
        arr_tup = []
        for i in range(len(arr_idx)):
            arr = []
            mul = 1
            for j in range(batch_size):
                arr.append((digits[arr_idx[i][j]][j]))
                x = confs[arr_idx[i][j]][j]
                mul *= x
            arr_tup.append((arr, mul))
        arr_tup.sort(key=lambda ele: ele[1], reverse=True)
        return arr_tup[0:Depth_Limit]

    def convert3to12(self, arr):
        bit_str = ""
        val = arr[0]
        binary = bin(val).replace("0b", "").zfill(3)
        bit_str += binary
        val = arr[1]
        binary = bin(val).replace("0b", "").zfill(3)
        bit_str += binary
        val = arr[2]
        binary = bin(val).replace("0b", "").zfill(3)
        bit_str += binary

        val = arr[3]
        binary = bin(val).replace("0b", "").zfill(3)
        bit_str += binary

        v = bit_str[0:12:1]
        digit = int(v, 2)
        return digit

    def convert12to3(self, val):
        ret = []
        binary = bin(val).replace("0b", "").zfill(12)
        v = binary[0:3:1]
        digit = int(v, 2)
        ret.append(digit)
        v = binary[3:6:1]
        digit = int(v, 2)
        ret.append(digit)
        v = binary[6:9:1]
        digit = int(v, 2)
        ret.append(digit)
        v = binary[9:12:1]
        digit = int(v, 2)
        ret.append(digit)
        return ret



    def calculate(self, digits):
        checksum = self.calculator.checksum(bytes(digits))
        return self.convert12to3(checksum)


    def verify(self, digits, confs):
        arr_tup = self.rank(digits, confs)
        batch_size = self.data_blck_size + self.csum_blck_size
        curDepth=0

        for arr, _ in arr_tup:
            data = arr[:-self.csum_blck_size]
            csum = arr[-self.csum_blck_size:]

            if self.calculator.verify(bytes(data), self.convert3to12(csum)):
                return data,csum,True,curDepth

            elif(curDepth>Depth_Limit):
                break

            curDepth+=1


        return arr_tup[0][0][:-self.csum_blck_size], arr_tup[0][0][-self.csum_blck_size:], False, 0


    def encode(self, digits):
        rem = len(digits) % self.data_blck_size
        if rem:
            digits = digits + [0] * (self.data_blck_size - rem)
        ret = []
        for i in range(0, len(digits), self.data_blck_size):
            d = digits[i:i + self.data_blck_size]
            e = self.calculate(d)
            ret += e
        return ret

    def combine(self, digits, confs):
        data_val = [digit[:-self.csum_arr_size] for digit in digits]
        csum_val = [digit[-self.csum_arr_size:] for digit in digits]
        data_conf = [conf[:-self.csum_arr_size] for conf in confs]
        csum_conf = [conf[-self.csum_arr_size:] for conf in confs]
        rem = len(data_val[0]) % self.data_blck_size
        if rem:
            for r in range(self.top_k):
              data_val[r] = data_val[r] + [0] * (self.data_blck_size - rem)
              data_conf[r] = data_conf[r] + [100] * (self.data_blck_size - rem)

        digits_alt = [[] for _ in range(self.top_k)]
        confs_alt = [[] for _ in range(self.top_k)]
        i = 0
        j = 0
        while i < len(data_val[0]):
            d = [dv[i:i + self.data_blck_size] for dv in data_val]
            d_conf = [dc[i:i + self.data_blck_size] for dc in data_conf]
            c = [cv[j:j + self.csum_blck_size] for cv in csum_val]
            c_conf = [cc[j:j + self.csum_blck_size] for cc in csum_conf]
            for k in range(self.top_k):
                digits_alt[k] += d[k] + c[k]
                confs_alt[k] += d_conf[k] + c_conf[k]
            i += self.data_blck_size
            j += self.csum_blck_size
        return digits_alt, confs_alt

    def recover(self, digits, confs):
        digits_alt, confs_alt = self.combine(digits, confs)
        dat = []
        csum = []
        batch_size = self.data_blck_size + self.csum_blck_size
        cnt = 0
        total_depth=0
        batch_count=0
        for i in range(0, len(digits_alt[0]), batch_size):
            d = [arr[i:i + batch_size] for arr in digits_alt]
            c = [arr[i:i + batch_size] for arr in confs_alt]
            x, y, b, curDepth = self.verify(d, c)
            dat += x
            csum += y
            batch_count+=1
            total_depth+=curDepth

            if not b:
                cnt += 1
            print("---",i)
        avg_depth= total_depth/batch_count
        return dat[:self.data_arr_size], csum[:self.csum_arr_size], cnt, avg_depth

############################ INITIALIZING PARAMETERS
top_1 = 95
data_blck_size=4 #variable
csum_blck_size=4  # fixed
DataChunk = len(Data)
checksumChunk= int(math.ceil(DataChunk/data_blck_size))*csum_blck_size
top_k=4
Depth_Limit = 450
#########################################################

dr = DigitRecover(data_blck_size, csum_blck_size, DataChunk, checksumChunk, top_k,Depth_Limit)
Checksum = dr.encode(Data)


if(checksumChunk!=len(Checksum)):
  print("ERROR in initializing the Checksum Size")


print(Data)
print('Size of array of  sender for Data: ',DataChunk)

print(Checksum)
print('Size of array of  sender for Checksum: ',len(Checksum))

[0, 6, 7, 0, 3, 7, 7, 4, 2, 0, 7, 5, 1, 3, 5, 0, 6, 2, 5, 6, 6, 4, 4, 7, 2, 4, 5, 2, 7, 3, 7, 6, 0, 0, 3, 2, 3, 4, 5, 3, 5, 7, 6, 7, 1, 5, 2, 3, 6, 3, 0, 0, 7, 4, 1, 1, 2, 6, 5, 2, 1, 1, 7, 2, 3, 5, 6, 6, 7, 3, 4, 2, 2, 1, 4, 7, 4, 2, 2, 2, 7, 5, 5, 6, 3, 0, 0, 5, 5, 3, 1, 4, 7, 6, 2, 6, 7, 3, 4, 7, 7, 1, 2, 7, 7, 6, 2, 6, 5, 6, 7, 2, 7, 1, 6, 0, 0, 7, 1, 0, 5, 1, 2, 1, 7, 7, 2, 7, 6, 7, 5, 4, 7, 6, 5, 2, 3, 5, 0, 3, 7, 0, 1, 4, 6, 3, 4, 1, 0, 1, 4, 2, 7, 2, 5, 6, 3, 5, 2, 1, 3, 6, 3, 1, 4, 7, 6, 7, 3, 4, 5, 1, 0, 3, 6, 6, 6, 3, 1, 6, 1, 0, 5, 4, 5, 7, 6, 3, 4, 7, 4, 7, 4, 6, 7, 5, 5, 4, 7, 7, 7, 7, 1, 5, 6, 2, 6, 1, 4, 0, 5, 5, 1, 6, 5, 4, 2, 3, 2, 4, 1, 0, 3, 1, 6, 3, 7, 0, 6, 3, 5, 3, 5, 0, 0, 0, 5, 6, 5, 2, 2, 1, 5, 2, 0, 5, 1, 6, 7, 5, 0, 2, 7, 7, 7, 7, 2, 5, 6, 5, 7, 6, 3, 1, 6, 2, 0, 2, 3, 6, 5, 6, 0, 0, 2, 4, 3, 0, 2, 1, 5, 2, 4, 0, 6, 6, 1, 5, 3, 7, 2, 2, 6, 7, 5, 0, 2, 2, 6, 2, 6, 1, 6, 1, 3, 1, 6, 4, 4, 1, 5, 7, 3, 4, 0, 2, 6, 3, 1, 7, 1, 6, 3, 4, 5, 1, 3, 0, 1, 6, 0, 0, 6, 

In [6]:
import copy


temp= copy.deepcopy(Data)
Data = Data + Checksum


print(Data)
print('Size of array of  sender for Data: ',len(Data))
print(len(temp))


[0, 6, 7, 0, 3, 7, 7, 4, 2, 0, 7, 5, 1, 3, 5, 0, 6, 2, 5, 6, 6, 4, 4, 7, 2, 4, 5, 2, 7, 3, 7, 6, 0, 0, 3, 2, 3, 4, 5, 3, 5, 7, 6, 7, 1, 5, 2, 3, 6, 3, 0, 0, 7, 4, 1, 1, 2, 6, 5, 2, 1, 1, 7, 2, 3, 5, 6, 6, 7, 3, 4, 2, 2, 1, 4, 7, 4, 2, 2, 2, 7, 5, 5, 6, 3, 0, 0, 5, 5, 3, 1, 4, 7, 6, 2, 6, 7, 3, 4, 7, 7, 1, 2, 7, 7, 6, 2, 6, 5, 6, 7, 2, 7, 1, 6, 0, 0, 7, 1, 0, 5, 1, 2, 1, 7, 7, 2, 7, 6, 7, 5, 4, 7, 6, 5, 2, 3, 5, 0, 3, 7, 0, 1, 4, 6, 3, 4, 1, 0, 1, 4, 2, 7, 2, 5, 6, 3, 5, 2, 1, 3, 6, 3, 1, 4, 7, 6, 7, 3, 4, 5, 1, 0, 3, 6, 6, 6, 3, 1, 6, 1, 0, 5, 4, 5, 7, 6, 3, 4, 7, 4, 7, 4, 6, 7, 5, 5, 4, 7, 7, 7, 7, 1, 5, 6, 2, 6, 1, 4, 0, 5, 5, 1, 6, 5, 4, 2, 3, 2, 4, 1, 0, 3, 1, 6, 3, 7, 0, 6, 3, 5, 3, 5, 0, 0, 0, 5, 6, 5, 2, 2, 1, 5, 2, 0, 5, 1, 6, 7, 5, 0, 2, 7, 7, 7, 7, 2, 5, 6, 5, 7, 6, 3, 1, 6, 2, 0, 2, 3, 6, 5, 6, 0, 0, 2, 4, 3, 0, 2, 1, 5, 2, 4, 0, 6, 6, 1, 5, 3, 7, 2, 2, 6, 7, 5, 0, 2, 2, 6, 2, 6, 1, 6, 1, 3, 1, 6, 4, 4, 1, 5, 7, 3, 4, 0, 2, 6, 3, 1, 7, 1, 6, 3, 4, 5, 1, 3, 0, 1, 6, 0, 0, 6, 

#Initialization of Accuracy


In [7]:

top_2= top_1+(100-top_1)/2.0
top_3= top_2+ (100-top_2)/2.0
top_4= top_3+ (100-top_3)/2.0


sender_val=copy.deepcopy(Data)

Total_Extracted_Val=[]
Total_R_Up= []
Total_R2_UP=[]
Total_R3_UP=[]


for i in range(DataChunk):
    Total_Extracted_Val.append(sender_val[i])
    Total_R_Up.append((sender_val[i]+1)%8)
    Total_R2_UP.append((sender_val[i]+2)%8)
    Total_R3_UP.append((sender_val[i]+3)%8)



for i in range(DataChunk, DataChunk+checksumChunk,1):
    Total_Extracted_Val.append(sender_val[i])
    Total_R_Up.append((sender_val[i]+1)%4)
    Total_R2_UP.append((sender_val[i]+2)%4)
    Total_R3_UP.append((sender_val[i]+3)%4)



#For Top-1

In [8]:
import math
error_to_be_occured_data= int(math.floor(DataChunk*(100-top_1)/100))
print("error introduced in data: ",error_to_be_occured_data, "\nData Accuracy Calculated: ",(DataChunk-error_to_be_occured_data)*100/DataChunk)

random.seed(12)
error_Index_1=[]
while(True):
     y = random.randrange(DataChunk)
     if y and y+1 not in error_Index_1:
        error_Index_1.append(y)
        error_Index_1.append(y+1)
     if(len(error_Index_1) == error_to_be_occured_data):
       break

print("Index length:", len(error_Index_1))

for i in range(DataChunk):
  if i in error_Index_1:
    Total_Extracted_Val[i] = (Total_Extracted_Val[i]+1)%8



#For Checksum

error_to_be_occured_chksum= int(math.floor(checksumChunk*(100-top_1)/100))
print("error introduced in Checksum: ",error_to_be_occured_chksum, "\nChecksum Accuracy Calculated: ",(checksumChunk-error_to_be_occured_chksum)*100/checksumChunk)

random.seed(12)
error_Index_1c=[]
while(True):
     y = random.randrange(DataChunk, DataChunk+checksumChunk)
     if y not in error_Index_1c:
        error_Index_1c.append(y)
     if(len(error_Index_1c) == error_to_be_occured_chksum):
       break

print("Index length:", len(error_Index_1c))

for i in range(DataChunk, DataChunk+checksumChunk):
  if i in error_Index_1c:
    Total_Extracted_Val[i] = (Total_Extracted_Val[i]+1)%4



error introduced in data:  500 
Data Accuracy Calculated:  95.0
Index length: 500
error introduced in Checksum:  500 
Checksum Accuracy Calculated:  95.0
Index length: 500


In [9]:
errorCount_D = int(math.floor((DataChunk*top_2/100) - (DataChunk*top_1/100)))

print(error_to_be_occured_data)
print(errorCount_D)


for i in range(errorCount_D):
    Total_R_Up[error_Index_1[i]] = sender_val[error_Index_1[i]]

errorCount_CK = int(math.floor((checksumChunk*top_2/100) - (checksumChunk*top_1/100)))

print(error_to_be_occured_chksum)
print(errorCount_CK)


for i in range(errorCount_CK):
    Total_R_Up[error_Index_1c[i]] = sender_val[error_Index_1c[i]]





500
250
500
250


In [10]:
errorCount_D2 = int(math.floor((DataChunk*top_3/100) - (DataChunk*top_2/100)))

print(error_to_be_occured_data)
print(errorCount_D2)

for i in range(errorCount_D,errorCount_D+errorCount_D2):
    Total_R2_UP[error_Index_1[i]] = sender_val[error_Index_1[i]]


errorCount_CK2 = int(math.floor((checksumChunk*top_3/100) - (checksumChunk*top_2/100)))

print(error_to_be_occured_chksum)
print(errorCount_CK2)



for i in range(errorCount_CK,errorCount_CK+errorCount_CK2):
    Total_R2_UP[error_Index_1c[i]] = sender_val[error_Index_1c[i]]

500
125
500
125


In [11]:
errorCount_D3 = int(math.floor((DataChunk*top_4/100) - (DataChunk*top_3/100)))

print(error_to_be_occured_data)
print(errorCount_D3)

for i in range(errorCount_D+errorCount_D2,errorCount_D+errorCount_D2+errorCount_D3):
    Total_R3_UP[error_Index_1[i]] = sender_val[error_Index_1[i]]


errorCount_CK3 = int(math.floor((checksumChunk*top_4/100) - (checksumChunk*top_3/100)))

print(error_to_be_occured_chksum)
print(errorCount_CK3)



for i in range(errorCount_CK+errorCount_CK2,errorCount_CK+errorCount_CK2+errorCount_CK3):
    Total_R3_UP[error_Index_1c[i]] = sender_val[error_Index_1c[i]]

500
62
500
62


#Retrived Data

In [12]:

print("Top-1: ",top_1,"\nTop-2: ",top_2, "\nTop-3: ", top_3, "\nTop-4: ",top_4)

print(len(Total_Extracted_Val), len(Total_R_Up),len(Total_R2_UP),len(Total_R3_UP))


incorrect=0
for i in range(len(sender_val)):
  if(sender_val[i]!=Total_Extracted_Val[i]):
    incorrect+=1

print('Total incorrect before decoding: ',incorrect)
TOP1_accuracy= (len(sender_val)-incorrect)/len(sender_val)
TOP1_ErrorCount = incorrect
print('Extraction Top-1 Accuracy: ',TOP1_accuracy)



correct=0
for i in range(len(sender_val)):
  if(sender_val[i]==Total_Extracted_Val[i] or sender_val[i]==Total_R_Up[i]):
    correct+=1

print('Extraction Top-2 Accuracy: ',(correct)/len(sender_val))

correct=0
for i in range(len(sender_val)):
  if(sender_val[i]==Total_Extracted_Val[i] or sender_val[i]==Total_R_Up[i] or sender_val[i]==Total_R2_UP[i]):
    correct+=1

print('Extraction Top-3 Accuracy: ',(correct)/len(sender_val))


correct=0
for i in range(len(sender_val)):
  if(sender_val[i]==Total_Extracted_Val[i] or sender_val[i]==Total_R_Up[i] or sender_val[i]==Total_R2_UP[i] or sender_val[i]==Total_R3_UP[i]):
    correct+=1

print('Extraction Top-4 Accuracy: ',(correct)/len(sender_val))




Top-1:  95 
Top-2:  97.5 
Top-3:  98.75 
Top-4:  99.375
20000 20000 20000 20000
Total incorrect before decoding:  999
Extraction Top-1 Accuracy:  0.95005
Extraction Top-2 Accuracy:  0.97505
Extraction Top-3 Accuracy:  0.98755
Extraction Top-4 Accuracy:  0.99375


#Block Accuracy

In [13]:
def match_(arr1, arr2):
  length= min(len(arr1),len(arr2))
  for i in range(length):
    if arr1[i] != arr2[i]:
      return False
  return True



correct=0
index = DataChunk
for i in range(0,DataChunk,data_blck_size):
  if(match_(sender_val[i:i+data_blck_size],Total_Extracted_Val[i:i+data_blck_size]) and match_(sender_val[index:index+csum_blck_size],Total_Extracted_Val[index:index+csum_blck_size])):
    correct+=1
  index+=csum_blck_size


print('Total correct data block: ',correct)
total_data_block = DataChunk/data_blck_size
print("(",data_blck_size+csum_blck_size,")"," block accuracy: ",correct/total_data_block*100)


Total correct data block:  1978
( 8 )  block accuracy:  79.12


In [14]:
Winner_confidence=[]
Runner_UP_confidence=[]
second_Runner_UP_confidence=[]
third_Runner_UP_confidence=[]

top1_confidence = top_1 * 10
top2_confidence = (top_2 - top_1)*10
top3_confidence = (top_3 - top_2)*10
top4_confidence = (top_4 - top_3)*10

print(top1_confidence,top2_confidence,top3_confidence,top4_confidence)


for i in range(0, DataChunk+checksumChunk):
  Winner_confidence.append(top1_confidence)
  Runner_UP_confidence.append(top2_confidence)
  second_Runner_UP_confidence.append(top3_confidence)
  third_Runner_UP_confidence.append(top4_confidence)


950 25.0 12.5 6.25


#Data Extraction

In [15]:
digits=[]
confs=[]

digits.append(Total_Extracted_Val)
digits.append(Total_R_Up)
digits.append(Total_R2_UP)
if(top_k>3):
  digits.append(Total_R3_UP)

confs.append(Winner_confidence)
confs.append(Runner_UP_confidence)
confs.append(second_Runner_UP_confidence)
if(top_k>3):
  confs.append(third_Runner_UP_confidence)


receiver_data, receiver_Checksum, failedExtraction,avg_depth = dr.recover(digits, confs)
print(len(receiver_data), len(receiver_Checksum))
print("Failed to match checksum and put top-1 : ",failedExtraction," times")
print("Average depth: ",avg_depth)



--- 0
--- 8
--- 16
--- 24
--- 32
--- 40
--- 48
--- 56
--- 64
--- 72
--- 80
--- 88
--- 96
--- 104
--- 112
--- 120
--- 128
--- 136
--- 144
--- 152
--- 160
--- 168
--- 176
--- 184
--- 192
--- 200
--- 208
--- 216
--- 224
--- 232
--- 240
--- 248
--- 256
--- 264
--- 272
--- 280
--- 288
--- 296
--- 304
--- 312
--- 320
--- 328
--- 336
--- 344
--- 352
--- 360
--- 368
--- 376
--- 384
--- 392
--- 400
--- 408
--- 416
--- 424
--- 432
--- 440
--- 448
--- 456
--- 464
--- 472
--- 480
--- 488
--- 496
--- 504
--- 512
--- 520
--- 528
--- 536
--- 544
--- 552
--- 560
--- 568
--- 576
--- 584
--- 592
--- 600
--- 608
--- 616
--- 624
--- 632
--- 640
--- 648
--- 656
--- 664
--- 672
--- 680
--- 688
--- 696
--- 704
--- 712
--- 720
--- 728
--- 736
--- 744
--- 752
--- 760
--- 768
--- 776
--- 784
--- 792
--- 800
--- 808
--- 816
--- 824
--- 832
--- 840
--- 848
--- 856
--- 864
--- 872
--- 880
--- 888
--- 896
--- 904
--- 912
--- 920
--- 928
--- 936
--- 944
--- 952
--- 960
--- 968
--- 976
--- 984
--- 992
--- 1000
--- 10

# Data Accuracy Using Checksum

In [23]:
incorrect=0
for i in range(DataChunk):
  if(sender_val[i]!=Total_Extracted_Val[i]):
    incorrect+=1

print('Total incorrect BEFORE DECODING (data only): ',incorrect,'\nData Extraction Accuracy: ',(DataChunk-incorrect)/DataChunk)
print("---------------------------------------------------")

DataErrorInitial= incorrect

incorrect=0
for i in range(DataChunk):
  if(sender_val[i]!=receiver_data[i]):
    incorrect+=1

DataErrorafterDecoding= incorrect
print('Total incorrect AFTER DECODING (data only): ',incorrect,'\nData Extraction Accuracy: ',(DataChunk-incorrect)/DataChunk)
print()
totalCorrectData = len(receiver_data)-incorrect

print("Data segment size: ",len(receiver_data))
print("Total correct Data: ",totalCorrectData,"(",totalCorrectData/DataChunk*100,"%)")
DataErrorCorrected = DataErrorInitial - DataErrorafterDecoding

Total incorrect BEFORE DECODING (data only):  499 
Data Extraction Accuracy:  0.9501
---------------------------------------------------
Total incorrect AFTER DECODING (data only):  252 
Data Extraction Accuracy:  0.9748

Data segment size:  10000
Total correct Data:  9748 ( 97.48 %)


#Cell Accuracy

In [24]:
wrong = incorrect # initializing the error occured in the data segment

index=0
for i in range(DataChunk,DataChunk+checksumChunk,1):
  if(sender_val[i]!=receiver_Checksum[index]):
    wrong+=1
  index+=1

ChecksumErrorInitial = TOP1_ErrorCount - DataErrorInitial
ChecksumErrorafterDecoding = wrong - incorrect
ChecksumErrorCorrected = ChecksumErrorInitial - ChecksumErrorafterDecoding

print('Total incorrect BEFORE DECODING (Data+Checksum): ',TOP1_ErrorCount,'\nData+Checksum TOP-1 Accuracy: ',TOP1_accuracy)
print("-----------------------------------------------------------")
print('Total incorrect AFTER DECODING (Data+Checksum): ',wrong,'\nData+Checksum Extraction Accuracy: ',(len(sender_val)-wrong)/len(sender_val))
print()
#print(sender_val[0:DataChunk])
#print(receiver_data)
print("Whole data(",DataChunk,")","+checksum(",checksumChunk,") segment size:",len(Total_Extracted_Val))


Total incorrect BEFORE DECODING (Data+Checksum):  999 
Data+Checksum TOP-1 Accuracy:  0.95005
-----------------------------------------------------------
Total incorrect AFTER DECODING (Data+Checksum):  476 
Data+Checksum Extraction Accuracy:  0.9762

Whole data( 10000 ) +checksum( 10000 ) segment size: 20000


#Debugging Checksum

In [25]:
#Initial checksum after querying the model

InitialExtractedChecksum = copy.deepcopy(Total_Extracted_Val[DataChunk:])
InitialReceiverChecksum=[]

for i in range(0,checksumChunk,4):
      val = dr.convert3to12(InitialExtractedChecksum[i:i+4])
      InitialReceiverChecksum.append(val)

print(len(InitialReceiverChecksum))

#receiver side checksum after correcting using confidence

Corrected_Checksum=[]

for i in range(0,checksumChunk,4):
      val = dr.convert3to12(receiver_Checksum[i:i+4])
      Corrected_Checksum.append(val)

print(len(Corrected_Checksum))



#Sender Side checksum array

enc=[]

for i in range(0,checksumChunk,4):
      val = dr.convert3to12(Checksum[i:i+4])
      enc.append(val)

print(len(enc))



2500
2500
2500


In [26]:
from tensorflow.python.ops.check_ops import assert_greater_equal_v2

count1 = 0 # CASE: 1  : initial checksum and Data both correct
count2 = 0 # CASE: 2  : error in either, but successfully corrected
count3 = 0 # CASE: 2  : error in either, but successfully corrected by returning top-1
count4 = 0 # CASE 3(a): error in either, but NOT corrected successfully (matched checksum with incorrect data)
count5 = 0 # CASE 3(b): error in either, but NOT corrected successfully (could not match within available choices)


array1 = []
array2 = []
array3 = []
array4 = []
array5 = []


# rem = len(sender_val) % data_blck_size
# if rem:
#   sender_val = sender_val + [0] * (data_blck_size - rem)

print(len(enc),len(sender_val),len(receiver_data)+len(receiver_Checksum))


total_nonmatched = 0

def counterror(arr1, arr2):
  length= min(len(arr1),len(arr2))
  t = 0
  for i in range(length):
    if arr1[i] != arr2[i]:
      t += 1
  return t

def match_(arr1, arr2):
  length= min(len(arr1),len(arr2))
  for i in range(length):
    if arr1[i] != arr2[i]:
      return False
  return True

batch_size =  data_blck_size # Here we assume data block size 4


for i in range(len(enc)):
  if(enc[i] == InitialReceiverChecksum[i] and match_(sender_val[i * batch_size:(i + 1) * batch_size], Total_Extracted_Val[i * batch_size:(i + 1) * batch_size]) and enc[i] == Corrected_Checksum[i] and match_(sender_val[i * batch_size:(i + 1) * batch_size], receiver_data[i * batch_size:(i + 1) * batch_size])):
      count1 += 1 # CASE: 1 --> initial checksum and Data both correct
      array1.append(sender_val[i * batch_size:(i + 1) * batch_size])
      array1.append(receiver_data[i * batch_size:(i + 1) * batch_size])
      array1.append(Total_Extracted_Val[i * batch_size:(i + 1) * batch_size])
      array1.append(Total_R_Up[i * batch_size:(i + 1) * batch_size])
      array1.append(Total_R2_UP[i * batch_size:(i + 1) * batch_size])
      array1.append(Total_R3_UP[i * batch_size:(i + 1) * batch_size])

  elif(enc[i] == Corrected_Checksum[i] and match_(sender_val[i * batch_size:(i + 1) * batch_size], receiver_data[i * batch_size:(i + 1) * batch_size])):
      count2 += 1 # CASE: 2 --> error in either, but successfully corrected
      array2.append(sender_val[i * batch_size:(i + 1) * batch_size])
      array2.append(receiver_data[i * batch_size:(i + 1) * batch_size])
      array2.append(Total_Extracted_Val[i * batch_size:(i + 1) * batch_size])
      array2.append(Total_R_Up[i * batch_size:(i + 1) * batch_size])
      array2.append(Total_R2_UP[i * batch_size:(i + 1) * batch_size])
      array2.append(Total_R3_UP[i * batch_size:(i + 1) * batch_size])


  else:
    if(enc[i] != Corrected_Checksum[i] and not match_(sender_val[i * batch_size:(i + 1) * batch_size], receiver_data[i * batch_size:(i + 1) * batch_size])):
      count4 += 1 # CASE: 3 -->  error in either, but NOT corrected successfully
                  # (a) error in both data and checksum (matched a checksum with incorrect Data)
      total_nonmatched += counterror(sender_val[i * batch_size:(i + 1) * batch_size], receiver_data[i * batch_size:(i + 1) * batch_size])
      array4.append(sender_val[i * batch_size:(i + 1) * batch_size])
      array4.append(receiver_data[i * batch_size:(i + 1) * batch_size])
      array4.append(Total_Extracted_Val[i * batch_size:(i + 1) * batch_size])
      array4.append(Total_R_Up[i * batch_size:(i + 1) * batch_size])
      array4.append(Total_R2_UP[i * batch_size:(i + 1) * batch_size])
      array4.append(Total_R3_UP[i * batch_size:(i + 1) * batch_size])

    elif(not match_(sender_val[i * batch_size:(i + 1) * batch_size], receiver_data[i * batch_size:(i + 1) * batch_size])):
        count5 += 1 # CASE: 3 -->  error in either, but NOT corrected successfully
                  #(b) error in only Data (Could not match within available choices)
        total_nonmatched += counterror(sender_val[i * batch_size:(i + 1) * batch_size], receiver_data[i * batch_size:(i + 1) * batch_size])
        array5.append(sender_val[i * batch_size:(i + 1) * batch_size])
        array5.append(receiver_data[i * batch_size:(i + 1) * batch_size])
        array5.append(Total_Extracted_Val[i * batch_size:(i + 1) * batch_size])
        array5.append(Total_R_Up[i * batch_size:(i + 1) * batch_size])
        array5.append(Total_R2_UP[i * batch_size:(i + 1) * batch_size])
        array5.append(Total_R3_UP[i * batch_size:(i + 1) * batch_size])
    else:
        count3 += 1 # CASE: 2 --> error in either, but successfully corrected
                    # error in only checksum, no checksum matches but corrects data by Returning the top-1
        total_nonmatched += counterror(sender_val[i * batch_size:(i + 1) * batch_size], receiver_data[i * batch_size:(i + 1) * batch_size])
        array3.append(sender_val[i * batch_size:(i + 1) * batch_size])
        array3.append(receiver_data[i * batch_size:(i + 1) * batch_size])
        array3.append(Total_Extracted_Val[i * batch_size:(i + 1) * batch_size])
        array3.append(Total_R_Up[i * batch_size:(i + 1) * batch_size])
        array3.append(Total_R2_UP[i * batch_size:(i + 1) * batch_size])
        array3.append(Total_R3_UP[i * batch_size:(i + 1) * batch_size])


print("total checksum: ",len(enc))
print(count1, count2, count3, count4,count5)
print()
print("Checksum and Data both correct:                     ",count1,"(","{:.2%}".format(count1/len(enc)*1.0),")" )
print("Error in either, but successfully corrected:         ",count2,"(","{:.2%}".format((count2)/len(enc)*1.0),")" )
print("Error in either, but NOT successfully corrected:     ",count3+count4+count5,"(","{:.2%}".format((count3+count4+count5)/len(enc)*1.0),")" )


print("    *matched incorrect Data with incorrect checksum    ",count4,"(","{:.2%}".format(count4/len(enc)*1.0),")")
print("    *matched incorrect Data with correct checksum:     ",count5,"(","{:.2%}".format(count5/len(enc)*1.0),")")
print("    *matched correct Data with incorrect checksum:           ",count3," (","{:.2%}".format(count3/len(enc)*1.0),")")

print("\n\nTotal Correct block: ",count1+count2,"(","{:.2%}".format((count1+count2)/len(enc)*1.0),")" )

print("Average error in non successful block: ", total_nonmatched/(count3+count4+count5))


2500 20000 20000
total checksum:  2500
1978 337 48 131 6

Checksum and Data both correct:                      1978 ( 79.12% )
Error in either, but successfully corrected:          337 ( 13.48% )
Error in either, but NOT successfully corrected:      185 ( 7.40% )
    *matched incorrect Data with incorrect checksum     131 ( 5.24% )
    *matched incorrect Data with correct checksum:      6 ( 0.24% )
    *matched correct Data with incorrect checksum:            48  ( 1.92% )


Total Correct block:  2315 ( 92.60% )
Average error in non successful block:  1.3621621621621622


#Block Accuracy Using Checksum

In [27]:
correct=0
index = DataChunk
index2=0
for i in range(0,DataChunk,data_blck_size):
  if(match_(sender_val[i:i+data_blck_size],receiver_data[i:i+data_blck_size]) and match_(sender_val[index:index+csum_blck_size],receiver_Checksum[index2:index2+csum_blck_size])):
    #print(sender_val[i:i+data_blck_size])
    #print(Total_Extracted_Val[i:i+data_blck_size])
    correct+=1
  index+=csum_blck_size
  index2+=csum_blck_size


print('Total correct data block: ',correct)
total_data_block = int(math.ceil(DataChunk/data_blck_size))
print("(",data_blck_size+csum_blck_size,")"," block accuracy: ",correct/total_data_block*100)

Total correct data block:  2315
( 8 )  block accuracy:  92.60000000000001


In [28]:
count1 = 0 # CASE: 1  : initial checksum and Data both correct
count2 = 0 # CASE: 2  : error in either, but successfully corrected
count3 = 0 # CASE: 2  : error in either, but successfully corrected by returning top-1
count4 = 0 # CASE 3(a): error in either, but NOT corrected successfully (matched checksum with incorrect data)
count5 = 0 # CASE 3(b): error in either, but NOT corrected successfully (could not match within available choices)


# print("\n\n\nCASE---1:In intial Checksum and Block Data both correct: ",len(array1)/6,"\n\n")
# for i in range(1,len(array1),6):
#   print(array1[i-1],"sender val")
#   print(array1[i],"Decoded val")
#   print(array1[i+1],"Top-1 val")
#   print(array1[i+2],"Top-2 val")
#   print(array1[i+3],"Top-3 val")
#   print(array1[i+4],"Top-4 val")
#   print("*"*20)

# print("\n\n\nCASE---2: error in either, but successfully corrected",len(array2)/6,"\n\n")
# for i in range(1,len(array2),6):
#   print(array2[i-1],"sender val")
#   print(array2[i],"Decoded val")
#   print(array2[i+1],"Top-1 val")
#   print(array2[i+2],"Top-2 val")
#   print(array2[i+3],"Top-3 val")
#   print(array2[i+4],"Top-4 val")
#   print("*"*20)


print("\n\n\nCASE---2: error in either, but successfully corrected by returning top-1",len(array3)/6,"\n\n")
for i in range(1,len(array3),6):
  print(array3[i-1],"sender val")
  print(array3[i],"Decoded val")
  print(array3[i+1],"Top-1 val")
  print(array3[i+2],"Top-2 val")
  print(array3[i+3],"Top-3 val")
  print(array3[i+4],"Top-4 val")
  print("*"*20)


# print("\n\n\n CASE 3(a): error in either, but NOT corrected successfully")
# print("(matched checksum with incorrect data)",len(array4)/6,"\n\n")
# for i in range(1,len(array4),6):
#   print(array4[i-1],"sender val")
#   print(array4[i],"Decoded val")
#   print(array4[i+1],"Top-1 val")
#   print(array4[i+2],"Top-2 val")
#   print(array4[i+3],"Top-3 val")
#   print(array4[i+4],"Top-4 val")
#   print("*"*20)

print("\n\n\n CASE 3(b): Error in either, but NOT corrected successfully")
print("(could not match within available choices)",len(array5)/6,"\n\n")
for i in range(1,len(array5),6):
  print(array5[i-1],"sender val")
  print(array5[i],"Decoded val")
  print(array5[i+1],"Top-1 val")
  print(array5[i+2],"Top-2 val")
  print(array5[i+3],"Top-3 val")
  print(array5[i+4],"Top-4 val")
  print("*"*20)




CASE---2: error in either, but successfully corrected by returning top-1 48.0 


[5, 1, 4, 4] sender val
[5, 1, 4, 4] Decoded val
[5, 1, 4, 4] Top-1 val
[6, 2, 5, 5] Top-2 val
[7, 3, 6, 6] Top-3 val
[0, 4, 7, 7] Top-4 val
********************
[6, 3, 7, 7] sender val
[6, 3, 7, 7] Decoded val
[6, 3, 7, 7] Top-1 val
[7, 4, 0, 0] Top-2 val
[0, 5, 1, 1] Top-3 val
[1, 6, 2, 2] Top-4 val
********************
[0, 1, 6, 0] sender val
[0, 1, 6, 0] Decoded val
[0, 1, 6, 0] Top-1 val
[1, 2, 7, 1] Top-2 val
[2, 3, 0, 2] Top-3 val
[3, 4, 1, 3] Top-4 val
********************
[6, 4, 0, 5] sender val
[6, 4, 0, 5] Decoded val
[6, 4, 0, 5] Top-1 val
[7, 5, 1, 6] Top-2 val
[0, 6, 2, 7] Top-3 val
[1, 7, 3, 0] Top-4 val
********************
[2, 5, 4, 7] sender val
[2, 5, 4, 7] Decoded val
[2, 5, 4, 7] Top-1 val
[3, 6, 5, 0] Top-2 val
[4, 7, 6, 1] Top-3 val
[5, 0, 7, 2] Top-4 val
********************
[5, 3, 6, 3] sender val
[5, 3, 6, 3] Decoded val
[5, 3, 6, 3] Top-1 val
[6, 4, 7, 4] Top-2 val
[7, 5, 0, 5

In [29]:
top1_digit=0
top2_digit=0
top3_digit=0
top4_digit=0

for i in range(DataChunk):
  if(sender_val[i] == receiver_data[i]):
      if(receiver_data[i]==Total_Extracted_Val[i]):
        top1_digit+=1
      elif(receiver_data[i]==Total_R_Up[i]):
        top2_digit+=1
      elif(receiver_data[i]==Total_R2_UP[i]):
        top3_digit+=1
      elif(receiver_data[i]==Total_R3_UP[i]):
        top4_digit+=1

for i in range(DataChunk,DataChunk+checksumChunk):
  if(sender_val[i] == receiver_Checksum[i-DataChunk]):
        if(receiver_Checksum[i-DataChunk]==Total_Extracted_Val[i]):
          top1_digit+=1
        elif(receiver_Checksum[i-DataChunk]==Total_R_Up[i]):
          top2_digit+=1
        elif(receiver_Checksum[i-DataChunk]==Total_R2_UP[i]):
          top3_digit+=1
        elif(receiver_Checksum[i-DataChunk]==Total_R3_UP[i]):
          top4_digit+=1

print("Top1 digit: ",top1_digit,"\nTop2 digit: ",top2_digit,"\nTop3 digit: ",top3_digit,"\nTop4 digit: ",top4_digit)
print("total: ",top1_digit+top2_digit+top3_digit+top4_digit)

Top1 digit:  18969 
Top2 digit:  344 
Top3 digit:  140 
Top4 digit:  71
total:  19524
